In [32]:
import time
import uuid
from pathlib import Path
from log_utils import setup_logger
from core.audio_to_text.audio_to_text_processor import audio_to_text_processor
import os
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import sql

from core.post_processors.client_detector import detect_manager
from core.post_processors.llm_processing.objections_resolver import resolve_objections

In [7]:

folder_path = f"{Path.home()}/Documents/Аудио Бринекс/Brinex_in_2025_04/"
audio_files = list(Path(folder_path).glob("*"))

audio_files = [f for f in Path(folder_path).iterdir() if f.is_file()]

In [5]:
audio_files

[PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89196298789-20250411-174723-1744382843.21909797.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89179802318-20250416-133444-1744799684.22349925.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-83412570493-20250408-090718-1744092438.21536377.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89090000723-20250430-202033-1746033633.23720425.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89123168160-20250417-102927-1744874967.22420772.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89271122023-20250411-090402-1744351442.21836776.mp3'),
 PosixPath('/Users/igorlapin/Documents/Аудио Бринекс/Brinex_in_2025_04/in-07010101-89056633338-20250420-085521-1745128521.22673222.mp3'),
 PosixPath('/Users/igorlapin/Docum

In [14]:
DB_CONFIG = {
    'dbname': 'neiro-insight',
    'user': 'postgres',
    'password': '1510261105',
    'host': 'localhost',
    'port': '5432'
}
engine = create_engine(
    f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
)

In [29]:
from datetime import datetime


def run_pipeline(audio_file: Path):
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"

    start_time = time.time()
    data = []
    row_data = []
    file_uuid = uuid.uuid4()
    data.append({
        'id': file_uuid,
        'file_name': audio_file.name,
        'status': "NOT_PROCESSED",
        'created_at': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    })

    pd.DataFrame(data).to_sql(
        name='audio_dialog',
        con=engine,
        if_exists='append',
        index=False
    )

    result = audio_to_text_processor(audio_file)
    detect_manager(result)

    # resolve_objections(result)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Время выполнения: {execution_time:.4f} секунд")

    for r in result.items:
        row_data.append({
            'id': uuid.uuid4(),
            'audio_dialog_fk_id': file_uuid,
            'row_num': r.phrase_id,
            'row_text': r.text,
            'speaker_id': r.speaker_id,
            'start': r.start_time,
            'end': r.end_time,
            'has_swear_word': False,
            'has_greeting': False
        })
        if r.is_copy_line:
            continue
        print(f"{r.phrase_id} [{r.start_time:.2f} - {r.end_time:.2f}]: {r.speaker_id} : {r.text}")
    pd.DataFrame(row_data).to_sql(
        name='dialog_rows',
        con=engine,
        if_exists='append',
        index=False
    )

    for prop in result.objection_prop:
        print(f"Строка с возражением: {result.items[prop.objection_str - 1].text}")
        print(f"Строка с Обработкой возражения: {result.items[prop.manager_offer_str - 1].text}")
        print(f"Решилась ли проблема: {prop.was_resolved}")

In [30]:
audio_file = audio_files[0]
run_pipeline(audio_file)

2025-06-22 23:11:26 - core.audio_to_text.diarizer - INFO     - Diarizing in-07010101-89196298789-20250411-174723-1744382843.21909797.mp3
2025-06-22 23:11:34 - core.audio_to_text.diarizer - INFO     - Diarization complete. Segments: 152, Speakers: 2
2025-06-22 23:11:36 - core.audio_to_text.transcriber - INFO     - Loading MLX Whisper model: mlx-community/whisper-large-v3-turbo
2025-06-22 23:11:36 - core.audio_to_text.transcriber - INFO     - Loading MLX Whisper model: mlx-community/whisper-large-v3-turbo
2025-06-22 23:11:36 - core.audio_to_text.transcriber - INFO     - Loading MLX Whisper model: mlx-community/whisper-large-v3-turbo


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 85163.53it/s]


Время выполнения: 127.3257 секунд
1 [0.00 - 4.80]: SALES : Добрый день, меня зовут Елена. Как могу к вам обращаться? 
2 [4.80 - 7.50]: CLIENT : Добрый день, меня зовут Ельнар. 
3 [7.50 - 10.60]: SALES : Могу помочь? 
4 [10.60 - 19.10]: CLIENT : Девушка, вопрос следующий. Сейчас были колеса даром по городу Казани.  lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg lleg

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "has_greeting_word" of relation "dialog_rows" does not exist
LINE 1: ...w_text, speaker_id, start, "end", has_swear_word, has_greeti...
                                                             ^

[SQL: INSERT INTO dialog_rows (id, audio_dialog_fk_id, row_num, row_text, speaker_id, start, "end", has_swear_word, has_greeting_word) VALUES (%(id__0)s, %(audio_dialog_fk_id__0)s, %(row_num__0)s, %(row_text__0)s, %(speaker_id__0)s, %(start__0)s, %(end__0) ... 16766 characters truncated ... , %(speaker_id__96)s, %(start__96)s, %(end__96)s, %(has_swear_word__96)s, %(has_greeting_word__96)s)]
[parameters: {'has_swear_word__0': False, 'row_num__0': 1, 'id__0': UUID('e82eaf3a-2588-4706-ab60-a275a315ef97'), 'has_greeting_word__0': False, 'audio_dialog_fk_id__0': UUID('cab832db-0d35-48e8-bbc3-63846a1b1962'), 'end__0': 4.8, 'start__0': 0.0, 'row_text__0': 'Добрый день, меня зовут Елена. Как могу к вам обращаться? ', 'speaker_id__0': 'SALE